In [1]:
#DPR is designed to wade through large and complex text databases to retrieve accurate info.
#Finds relevant text passages from data.

In [2]:
questions = [
    "What is the capital city of England?",
    "What is the best sci-fi movie saga?",
    "How many times do I love you?"
]

contexts = [
    "London is the capital city of England.",
    "I love you 3000 times.",
    "The best sci-fi movie saga is Star Wars.",
    
    "What is the capital city of England?",
    "The capital city of Türkiye is Ankara.",
    "What is the best sci-fi movie saga?",
    "sci-fi movies are being watched by millions.",
    "Do I really love you?"
]

In [3]:
import torch
from transformers import DPRContextEncoderTokenizer, DPRContextEncoder, DPRQuestionEncoderTokenizer, DPRQuestionEncoder

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

ctx_name = 'facebook/dpr-ctx_encoder-single-nq-base'
ctx_model = DPRContextEncoder.from_pretrained(ctx_name).to(device)
ctx_tokenizer = DPRContextEncoderTokenizer.from_pretrained(ctx_name)

question_name = 'facebook/dpr-question_encoder-single-nq-base'
question_model = DPRQuestionEncoder.from_pretrained(question_name).to(device)
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained(question_name)

config.json:   0%|          | 0.00/492 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

c:\Users\radio\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()
Some weights of the model checkpoint at facebook/dpr-ctx_encoder-single-nq-base were not used when initializing DPRContextEncoder: ['ctx_encoder.bert_model.pooler.dense.bias', 'ctx_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRContextEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRContextEncoder from the checkpoint of a model that you expect to be

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
xb_tokens = ctx_tokenizer(contexts, max_length = 256, padding = 'max_length', return_tensors='pt').to(device)
xb = ctx_model(**xb_tokens)

xq_tokens = ctx_tokenizer(questions, max_length = 256, padding = 'max_length', return_tensors='pt').to(device)
xq = question_model(**xq_tokens)

In [5]:
xq.keys()

odict_keys(['pooler_output'])

In [6]:
xq.pooler_output.shape, xb.pooler_output.shape

(torch.Size([3, 768]), torch.Size([8, 768]))

In [7]:
from sentence_transformers.util import cos_sim

for i, xq_vec in enumerate(xq.pooler_output):
    probs = cos_sim(xq_vec, xb.pooler_output)
    argmax = torch.argmax(probs)
    print(questions[i])
    print(contexts[argmax])
    print('-----')

What is the capital city of England?
London is the capital city of England.
-----
What is the best sci-fi movie saga?
The best sci-fi movie saga is Star Wars.
-----
How many times do I love you?
I love you 3000 times.
-----
